In [ ]:
# what is in this file:
# Cell 2: the imports that would need to be included
# Cell 3: a fuction provided for you (you may want to include it as a function in your code too in a fresh cell)
# Cell 4: an example for how to use / call the function to generate test data that you will need in week 11.

# you will need the code in cells 2, 3, and 4 for next week, but this is a preview for you.

# James will cover code in week 11 to create test data... James is going to show you some code in future weeks which can try and guess for you!



In [ ]:
# imports needed for 
import numpy as np
import matplotlib.pyplot as plt 
import math

import pandas as pd

from scipy import signal
# you don't need to copy the code above... just make sure it already exists somewhere in your grou project file.


In [ ]:
"""
pseudocode for the function...

def create_training_data_from(list of file names, file to output to):
	Create an empty training_set numpy array

    for each file in the list of file names
        import the data from the file

        For each activity in the imported data:
            get activity data into activity_data variable
            smooth over the data (remove noise)
            For each group of 1000 rows in that activity_data:
                Get the minimum, maximum and average of the values for 3 axis of the wrist accelerometer
                Store them in a temp_array
                Add the activity number to the end of the temp_array
			Add that temp_array to the end of the training_set
	
	Take the data in traning_set and save it to a file specified by the output

"""


def create_training_data_from_files(list_of_filenames, output_filename):

    #create the empty training set where we are going to add our "features"
    training_set = np.empty(shape=(0, 10))
    
    for dataset_file in list_of_filenames:

        #import the file contents into a panadas data frame
        imported_data = pd.read_csv(dataset_file, sep=',', header=None)

        #generate "features" for each activitiy
        for activityNumber in range(1,14):
            
            #get all data relating to that activity and convert to a numpy ndarray
            activity_data = imported_data[imported_data[24] == activityNumber].values

            #smooth over the data for columns 0, 1, 2, ...23 (not column 24)
            b, a = signal.butter(4, 0.04, 'low', analog=False)
            for j in range(24):
                activity_data[:, j] = signal.lfilter(b, a, activity_data[:, j])
            
            #how many full rows of 1000 are there for this activity data?
            number_of_training_samples = int( len(activity_data)/1000 )
            print(  "File " + dataset_file +
                    " has " + str(number_of_training_samples) + " samples of 1000 rows"+
                    "for activity: " + str(activityNumber))
            
            #for each sample of 1000 rows... scan the data and add the scan results to training_set
            for sample_number in range(number_of_training_samples):
                #sample data (get the next 1000 rows and all the columns)
                sample_data = activity_data[ 
                                1000 * sample_number : 1000 * (sample_number + 1) , 
                                :
                            ]
                #we are about to build up a feature_sample that will have 10 columns
                feature_sample = []
                #sample from file 4 in week 7 prac
                for i in range(3):
                    feature_sample.append(np.min(sample_data[:, i]))
                    feature_sample.append(np.max(sample_data[:, i]))
                    feature_sample.append(np.mean(sample_data[:, i]))
                # add the activtiy number (The last column from the row of data)
                feature_sample.append(int(sample_data[0, -1])) 
                #make it in to an ndarray so it can be added to training data
                feature_sample = np.array([feature_sample]) 
                training_set = np.concatenate((training_set, feature_sample), axis=0)
            
    #now save all this training data into a file to be used at a later date
    df_training = pd.DataFrame(training_set)
    df_training.to_csv(output_filename, index=None, header=None)
    print('attempted to create '+ output_filename +' ... check if the file was created!')
    print(str(len(training_set)) + " data rows should be in the output file")

       


In [ ]:
# you have downloaded 2 dataset files so far... but in future weeks, you may need to download a few more
# you can use the code below to say whay dataset files you want to use.

#
# Start: create training data
#

#start off with an empty list
filenames_to_use_for_training_data = []

# if you just want to use dataset_2.txt, then make it = [2]
# if you want to use dataset_1.txt up to and including dataset_5.txt datasets for training data, then make it = [1, 2, 3, 4, 5]
# if you want to use select ones, you can list the numbers in any order ... e.g. = [3, 5, 2, 7, 10]
# make sure you have the same 
dataset_file_numbers = [1, 2]

print("Dataset files being used for training data: " + str(dataset_file_numbers))

# this for loop generates the filenames of the datasets you want to use.
for number in dataset_file_numbers:
    filenames_to_use_for_training_data.append("dataset_"+str(number)+".txt")

# this this function call will generate a feature training set from your selected dataset files
# (See James lecture from week 7 and also from week 11 for what these training sets are for)
create_training_data_from_files(filenames_to_use_for_training_data, "training_data2.csv")


#
# End: create training data
#
